# The Db2 Data Management Console Technical Preview 

The Jupyter Notebook in the project contains examples of how to use the Open APIs and composable interface that is available in the Db2 Data Management Console Technical Preview. Everything in the User Interface is also available through an open and fully documented RESTful Services API. You can find the full documentation in the Help included with the Db2 Data Management Console.

The technical preview also lets you embed elements of the full user interface into an IFrame by constructing the appropriate URL. The sample notebooks include examples of how to do this.

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas.io.json import json_normalize

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, userid, password, profile=""):
        credentials = {'userid':userid, 'password':password}
        r = requests.post(self.url+'/auth/tokens', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = r.json()['token']
            if profile == "":
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken}
            else:
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken, 'X-DB-Profile': profile}
        else:
            print ('Unable to authenticate, no bearer token obtained')
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def getSchemas(self):
        return self.getRequest('/schemas')
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/sql_jobs/'+jobid)
    
    def getCurrentApplicationsConnections(self, includeSystem='true'):
        return self.getRequest('/metrics/applications/connections/current/list?&include_sys='+str(includeSystem))
    
    def getInflightCount(self, startTime, endTime):
        return self.getRequest('/metrics/statements/inflight_executions/current/list?start='+str(startTime)+'&end='+str(endTime));
       
    def getInflightCurrentList(self, includeSystem='true'):
        return self.getRequest('/metrics/statements/inflight_executions/current/list?'+'&include_sys='+str(includeSystem));
    
    def getIndividualStatementExecution(self, startTime, endTime, limit=100, includeSystem='false'):
        return self.getRequest('/metrics/statements/evmon_activity?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem)+'&offset=0&limit='+str(limit))

    def getFiles(self, path):
        return self.getRequest('/home'+path)
    
    def getUsers(self):
        return self.getRequest('/users')
    
    def getTablesMetrics(self, startTime, endTime, includeSystem='false'):
        return self.getRequest('/metrics/tables?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem));

    def getAverageResponseTime(self, startTime, endTime):
        return self.getRequest('/metrics/average_response_time?start='+str(startTime)+'&end='+str(endTime));    
    
    def getUnitsOfWork(self, startTime, endTime):
        return self.getRequest('/applications/uow?start='+str(startTime)+'&end='+str(endTime));    
    
    def getSchemaSize(self, startTime, endTime, tabSchema):
        return self.getRequest('/metrics/storage/schemas/'+tabSchema+'/timeseries?start='+str(startTime)+'&end='+str(endTime));
  
    def getSearchViewList(self, searchtext, show_systems="false"):
        return self.getRequest('/admin/schemas/obj_type/view?search_name='+searchtext+'&show_systems='+str(show_systems)+'&rows_return=200');
    
    def getSearchTableList(self, searchtext):
        return self.getRequest('/admin/schemas/obj_type/table?search_name='+searchtext+'&show_systems=true&rows_return=100');
              
    def getRowsRead(self, startTime, endTime):
        return self.getRequest('/metrics/rows_read?start='+str(startTime)+'&end='+str(endTime));

    def getResponseTime(self, startTime, endTime):
        return self.getRequest('/metrics/response_time?start='+str(startTime)+'&end='+str(endTime));

    def getStatementsCount(self, startTime, endTime):
        return self.getRequest('/metrics/statements_count?start='+str(startTime)+'&end='+str(endTime));

    def getPackageCacheStatement(self, startTime, endTime, show_systems='true'):
        return self.getRequest('/metrics/statements/package_cache?start='+str(startTime)+'&end='+str(endTime)+'&show_systems='+str(show_systems))
    
    def postSearchObjects(self, obj_type, search_text, rows_return=100, show_systems='false', is_ascend='true'):     
        json = {"search_name":search_text,"rows_return":rows_return,"show_systems":show_systems,"obj_type":obj_type,"filters_match":"ALL","filters":[]}       
        return self.postRequest('/admin/'+str(obj_type)+'s',json);
                
    def putFile(self, filename, path):
        with open(filename, 'rb') as f:
            r = requests.post(self.url+'/home_content/path', files={filename: f}, verify = self.verify, headers=self.headers, proxies = self.proxies)
            
    def getTablesInSchema(self, schema):
        return self.getRequest('/schemas/'+str(schema)+'/tables');
    
    def getTableStorageBySchema(self):
        return self.getRequest('/metrics/storage/schemas?end=0&include_sys=true&limit=1000&offset=0&start=0')
    
    def getCurrentPackageCacheList(self, show_systems="false"):
        return self.getRequest('/metrics/statements/package_cache/current/list?include_sys='+str(show_systems))
    
    def getProfile(self,profile):
        return self.getRequest('/dbprofiles/'+profile)    

In [3]:
# Setup data frame set calculation functions
def epochtotimeseries(epoch):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch/1000))
def KBtoGB(kb):
    return kb/1024/1024

In [4]:
# Setup time series calculation values
import time
from datetime import date
endTime = int(time.time())
endTime = endTime*1000
startTime = endTime-(600*1000)
oneWeek = 604800000

In [5]:
# Increase the width of the notebook working spaces
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
# Enterprise Console Local
profile = "Sample"
profileURL = "?profile="+profile
database = 'http://127.0.0.1:11090'
databaseAPI = Db2(database+'/dbapi/v4')
databaseAPI.authenticate("admin", "password", profile)

In [7]:
# List Monitoring Profile
r = databaseAPI.getProfile(profile)
json = databaseAPI.getJSON(r)
print(json)

{'name': 'Sample', 'HADR_Role': 'STANDARD', 'disableDataCollection': 'false', 'databaseVersion': '11.1.2', 'databaseName': 'SAMPLE', 'timeZone': '0', 'isHADR': 'false', 'DB2Instance': 'db2inst1', 'db2license': 'DEC', 'isInstPureScale': 'false', 'databaseVersion_VRMF': '11.1.2.2', 'sslConnection': 'false', 'userProfileRole': 'OWNER', 'timeZoneDiff': '-14400000', 'host': 'localhost', '_PROFILE_INIT_': 'true', 'dataServerType': 'DB2LUW', 'port': '50000', 'URL': 'jdbc:db2://localhost:50000/SAMPLE:retrieveMessagesFromServerOnGetMessage=true;', 'edition': 'DEC', 'isInstPartitionable': 'false', 'dataServerExternalType': 'DB2LUW', 'CF_NODE': '', 'capabilities': '["trial"]', 'OSType': 'Linux'}


In [8]:
# List all schema
r = databaseAPI.getSchemas()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json['resources']))
    print(', '.join(list(df)))
    display(df[['name']].head(10))
else:
    print(databaseAPI.getStatusCode(r))   

definertype, name


,name
0,DB2INST1
1,DB2OE
2,DB2OSC
3,DSJOBMGR
4,DSSCHED
5,DSSHSV1
6,DSWEBSECURITY
7,HEALTHMETRICS
8,IBMIOCM
9,IBMOTS


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/summary'+profileURL, width=1400, height=1000)

In [9]:
# Retrieve the number of rows read over the last week
# List the last 10 data points
# Graph the history
lastWeek = endTime - oneWeek
r = databaseAPI.getRowsRead(lastWeek, endTime)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:
        df = pd.DataFrame(json_normalize(json['timeseries']))
        print('Columns')
        print(', '.join(list(df)))
        df['timestamp'] = df['timestamp'].apply(epochtotimeseries)
        display(df[['timestamp','rows_read_per_min']].tail(20))
        df.plot(x='timestamp',y='rows_read_per_min') 
    else: 
        print('No data returned')
else:
    print(databaseAPI.getStatusCode(r))
    


Columns
interval, rows_read_per_min, timestamp


,timestamp,rows_read_per_min
146,2019-05-29 14:46:54,0.0
147,2019-05-29 14:51:54,0.0
148,2019-05-29 14:56:56,0.0
149,2019-05-29 15:01:55,0.0
150,2019-05-29 15:06:54,0.0
151,2019-05-29 15:11:55,0.0
152,2019-05-29 15:16:55,0.0
153,2019-05-29 15:21:55,0.0
154,2019-05-29 15:26:55,0.0
155,2019-05-29 15:31:57,0.0


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/overview'+profileURL, width=1600, height=1200)

In [ ]:
IFrame(database+'/console/?mode=compact#sql/explain/e015be3b-2849-4ed4-ab9a-478c1cea38bc', width=1600, height=700)

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/inflight_executions'+profileURL, width=1400, height=360)

In [10]:
# Retreive the current statements running now
# Display the top 10 by execution time
r = databaseAPI.getInflightCurrentList()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='exec_time_ms', ascending=False)
        display(df[['application_name','stmt_text','exec_time_ms','estimated_runtime_ms']].head(10))
    else: 
        print('No data returned')
else:
    code = databaseAPI.getStatusCode(r)
    databaseAPI.printResponse(r, code)

Columns
activity_id, activity_state, adm_bypassed, application_handle, application_name, client_ipaddr, effective_query_degree, entry_time, estimated_runtime_ms, exec_time_ms, idle_time_ms, max_estimated_shrheap_per_part_percent, max_shrheap_allocated_per_part_percent, max_shrheap_peak_per_part_percent, query_cost_estimate_timerons, queue_time_ms, rows_read, session_auth_id, stmt_text, uow_id


,application_name,stmt_text,exec_time_ms,estimated_runtime_ms
0,UC_MYMON,/* IBM_DSSNAP */ WITH unicode_fix(x) AS(\n\t\t...,0,5296740


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/individual'+profileURL, width=1400, height=480)

In [11]:
# Retrieve the statements that ran over the last week.
# Retrieve the first 10000 statements
# Show the top ten by Total CPU TIME
startTimeMinusWeek = endTime - oneWeek
r = databaseAPI.getIndividualStatementExecution(startTimeMinusWeek, endTime, 10000)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:     
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='total_cpu_time', ascending=False)
        display(df[['sql_text','total_cpu_time','wlm_queue_time_total','stmt_exec_time']].head(10))
    else: 
        print('No data returned')  
else:
    code = databaseAPI.getStatusCode(r)
    databaseAPI.printResponse(r, code)

Columns
activity_id, activity_type, address, agents_top, appl_id, appl_name, coord_partition_num, direct_reads, direct_writes, executable_id, fed_rows_deleted, fed_rows_inserted, fed_rows_read, fed_rows_updated, fed_wait_time, fed_waits_total, lock_wait_time, member, partial_record, prep_time, query_cost_estimate, rows_read, rows_read_per_returned, rows_returned, service_subclass_name, service_superclass_name, session_auth_id, sql_hash_id, sql_text, sqlcode, sqlwarn, stmt_exec_time, time_completed, time_started, total_act_wait_time, total_cpu_time, uow_id, wlm_queue_time_total, workloadname


,sql_text,total_cpu_time,wlm_queue_time_total,stmt_exec_time
50,select * from syscat.tables,15,0,31
20,select * from syscat.tables,14,0,26
14,select * from syscat.tables,14,0,21
22,select * from syscat.tables,13,0,18
24,select * from syscat.tables,13,0,20
16,select * from syscat.tables,13,0,37
48,select * from syscat.tables,12,0,19
23,select * from syscat.indexes,4,0,5
19,select * from syscat.indexes,3,0,6
15,select * from syscat.indexes,3,0,5


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/package_cache'+profileURL, width=1400, height=480)

In [12]:
# Retrieve the current package cache list 
# Show the first ten as sorted by the statement execution time
r = databaseAPI.getCurrentPackageCacheList("true")
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:  
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='stmt_exec_time_ms', ascending=False)
        display(df[['stmt_text','stmt_exec_time_ms','stmtid']].head(10))
    else: 
        print('No data returned')  
else:
    print(databaseAPI.getStatusCode(r))

Columns
_per_min, availabePreviousTimestamp, calFieldsMap, col_vector_consumer_overflows, collected, coord_stmt_exec_time_ms, dbcollectTimestamp, dbconn_int, deltaDone, estimated_runtime_ms, estimated_sort_shrheap_top, ext_table_read_volume_kb, ext_table_recv_volume_kb, ext_table_send_volume_kb, ext_table_write_volume_kb, fed_rows_deleted, fed_rows_inserted, fed_rows_read, fed_rows_updated, fed_wait_time_ms, fed_waits_total, groupKeyOnMembers, hash_grpby_overflows, hash_join_overflows, interval, key, level_tag, lock_escals, lock_wait_time_ms, lock_waits, logical_reads, num_exec_with_metrics, olap_func_overflows, physical_reads, planid, pool_data_l_reads, pool_index_l_reads, pool_read_time_ms, pool_write_time_ms, pool_writes, post_threshold_col_vector_consumers, post_threshold_hash_grpbys, post_threshold_hash_joins, post_threshold_olap_funcs, post_threshold_sorts, prefetch_wait_time_ms, rows_modified, rows_read, rows_returned, schemaName, semantic_env_id, sort_overflows, sort_shrheap_to

,stmt_text,stmt_exec_time_ms,stmtid
2,/* IBM_DSSNAP */ with unicode_fix(x) as (selec...,194,-2784068627434905255
12,/* IBM_DSSNAP */ set event monitor RTMON_EVMON...,9,7880021567290041286
13,/* IBM_DSSNAP */ select sum(fs_total_size)/(CA...,5,2688556594859344219
4,/* IBM_DSSNAP */ delete from (select CAST (? A...,3,-1293921616590792542
9,/* IBM_DSSNAP */ delete from (select CAST (? A...,3,507896686448705249
6,/* IBM_DSSNAP */ WITH unicode_fix(x) AS ( SELE...,2,-2755369640262110092
0,/* IBM_DSSNAP */ select event_mon_state(evmonn...,0,9142607139278373844
1,/* IBM_DSSNAP */ select count(CAST (? AS INT))...,0,4151764324223964662
3,"call SYSIBM.SQLCAMESSAGECCSID(?,?,?,?,?,?,?,?,...",0,8110287721056492873
5,/* IBM_DSSNAP */ WITH UNICODE_FIX(X) AS ( SELE...,0,-3421969322199409333


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/locked_objects_with_waiting_connections'+profileURL, width=1400, height=480)

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/connections'+profileURL, width=1400, height=480)

In [13]:
# Display the 10 most recently started Database Connections
r = databaseAPI.getCurrentApplicationsConnections()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0: 
        df = pd.DataFrame(json_normalize(json['resources']))
        print(', '.join(list(df)))
        df = df.sort_values(by='connection_start_time', ascending=False)
        df['connection_start_time'] = df['connection_start_time'].apply(epochtotimeseries)
        display(df[['application_name','application_handle','connection_start_time']].head(10))
    else: 
        print('No data returned')  
else:
    print(databaseAPI.getStatusCode(r))

act_aborted_total_per_min, act_completed_total_per_min, application_handle, application_name, client_ipaddr, client_userid, connection_start_time, ddl_sql_stmts_per_min, deadlocks_per_min, dynamic_sql_stmts_per_min, estimated_sort_shrheap_top_pages, failed_sql_stmts_per_min, idle_time_ms, max_estimated_shrheap_per_part_percent, max_shrheap_allocated_per_part_percent, max_shrheap_peak_per_part_percent, memory_pool_used_kb, other_stmts_per_min, rejected_rate, select_sql_stmts_per_min, service_superclass_name, session_auth_id, sort_shrheap_allocated_pages, sort_shrheap_top_pages, static_sql_stmts_per_min, thresh_violations_per_min, total_cpu_time_ms_per_min, uid_sql_stmts_per_min, uow_log_space_used_kb, workload_name, workload_occurrence_state


,application_name,application_handle,connection_start_time
0,UC_MYMON,8143,2019-05-29 16:26:36


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/wlm_workload_summary', width=1400, height=480)

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/storage', width=1400, height=480)

In [14]:
# Retrieve the storage size for the Historical Data Repostory stored in IBMCONSOLE schema.
# If this doesn't run against the repository database you will get a zero result set
# List the last 10 data points
# Graph the history of the storage usage
tabSchema = 'IBMCONSOLE'
OneWeekStartTime = endTime - oneWeek
r = databaseAPI.getSchemaSize(0, endTime, tabSchema)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    print(json)
    if json['count'] > 0: 
        df = pd.DataFrame(json_normalize(json['timeseries']))
        print('Columns')
        print(', '.join(list(df)))
        df['total_physical_size_gb'] = df['total_physical_size_kb'].apply(KBtoGB)
        df['timestamp'] = df['timestamp'].apply(epochtotimeseries)
        display(df[['timestamp','total_physical_size_gb']].tail(10))
        df.plot(x='timestamp',y='total_physical_size_gb')
    else: 
        print('No data returned') 
else:
    print(databaseAPI.getStatusCode(r))

{'timeseries': [], 'count': 0}
No data returned


In [15]:
# List storage used by schema
# Display the top ten schemas
r = databaseAPI.getTableStorageBySchema()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)        
    if json['count'] > 0: 
        df = pd.DataFrame(json_normalize(json['resources']))
        print(', '.join(list(df)))
        df['space_mb'] = df['data_physical_size_kb'].apply(lambda x: x / 1024)
        df = df.sort_values(by='data_physical_size_kb', ascending=False)    
        display(df[['tabschema','space_mb']].head(10))
    else: 
        print('No data returned') 
else:
    print("RC: "+str(databaseAPI.getStatusCode(r)))    

column_organized_data_logical_size_kb, column_organized_data_physical_size_kb, data_logical_size_kb, data_physical_size_kb, estimated_reclaim_size_kb, index_logical_size_kb, index_physical_size_kb, interval, lob_logical_size_kb, lob_physical_size_kb, long_logical_size_kb, long_physical_size_kb, tabschema, timestamp, total_logical_size_kb, total_physical_size_kb, xml_logical_size_kb, xml_physical_size_kb


,tabschema,space_mb
17,IBMIOCM,90.5000
0,SYSIBM,39.1875
19,IBM_RTMON_DATA,28.5000
12,DB2OSC,25.0000
14,IBM_RTMON,22.0000
15,DB2INST1,19.5000
13,SYSTOOLS,19.2500
18,HEALTHMETRICS,14.0000
16,IBMOTS,14.0000
6,IBMPDQ,11.0000


In [ ]:
# List any open alerts using part of the Console Interface
IFrame(database+'/console/?mode=compact#monitor/notification'+profileURL, width=1400, height=480)

In [ ]:
# Explore tables in your database using part of the Console Interface
IFrame(database+'/console/?mode=compact#explore/table'+profileURL, width=1400, height=480)

In [16]:
# Search for tables across all schemas that match simple search critera 
# Display the first 100
# Switch between searching tables or views
object = 'view'
#object = 'table'
r = databaseAPI.postSearchObjects(object,"TABLE",10,'true','false')
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print('Columns:')
    print(', '.join(list(df)))
    display(df[[object+'_name']].head(100))
else:
    print("RC: "+str(databaseAPI.getStatusCode(r)))

Columns:
alter_time, create_time, optimize_query, owner, owner_type, read_only, sql, stats_time, valid, view_check, view_name, view_schema


,view_name
0,TABLE_CONSTRAINTS
1,TABLES
2,TABLES_S
3,EVENTTABLES
4,TABLES
5,TABLESPACES
6,TABLES
7,SQLTABLEPRIVILEGES
8,SQLTABLETYPES
9,SQLTABLES


In [17]:
# Find all the tables in the SYSIBM schema and display the first 10
schema = 'SYSIBM'
r = databaseAPI.getTablesInSchema(schema)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json['resources']))
    print(', '.join(list(df)))
    display(df[['schema','name']].head(10))
else:
    print(databaseAPI.getStatusCode(r))   

name, schema


,schema,name
0,SYSIBM,SYSATTRIBUTES
1,SYSIBM,SYSAUDITEXCEPTIONS
2,SYSIBM,SYSAUDITPOLICIES
3,SYSIBM,SYSAUDITUSE
4,SYSIBM,SYSBUFFERPOOLNODES
5,SYSIBM,SYSBUFFERPOOLS
6,SYSIBM,SYSCHECKS
7,SYSIBM,SYSCODEPROPERTIES
8,SYSIBM,SYSCOLAUTH
9,SYSIBM,SYSCOLCHECKS


In [18]:
#Run an SQL Statement
r = databaseAPI.runSQL("select * from syscat.tables")
if (databaseAPI.getStatusCode(r)==201):
    jobid = databaseAPI.getJSON(r)['id']
    r = databaseAPI.getSQLJobResult(jobid)
    if (databaseAPI.getStatusCode(r)==200):
        pprint(databaseAPI.getJSON(r))
    else:
        print(databaseAPI.getStatusCode(r))

{'id': '1559161642136_2067242640', 'results': [], 'status': 'running'}


In [19]:
#If the results don't come back immediately then try again
r = databaseAPI.getSQLJobResult(jobid)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    print(json)
else:
    print(databaseAPI.getStatusCode(r))

{'id': '1559161642136_2067242640', 'results': [{'exceed_size': 'yes', 'rows_count': 11, 'runtime_seconds': 0.019999999552965164, 'columns': ['TABSCHEMA', 'TABNAME', 'OWNER', 'OWNERTYPE', 'TYPE', 'STATUS', 'BASE_TABSCHEMA', 'BASE_TABNAME', 'ROWTYPESCHEMA', 'ROWTYPENAME', 'CREATE_TIME', 'ALTER_TIME', 'INVALIDATE_TIME', 'STATS_TIME', 'COLCOUNT', 'TABLEID', 'TBSPACEID', 'CARD', 'NPAGES', 'MPAGES', 'FPAGES', 'NPARTITIONS', 'NFILES', 'TABLESIZE', 'OVERFLOW', 'TBSPACE', 'INDEX_TBSPACE', 'LONG_TBSPACE', 'PARENTS', 'CHILDREN', 'SELFREFS', 'KEYCOLUMNS', 'KEYINDEXID', 'KEYUNIQUE', 'CHECKCOUNT', 'DATACAPTURE', 'CONST_CHECKED', 'PMAP_ID', 'PARTITION_MODE', 'LOG_ATTRIBUTE', 'PCTFREE', 'APPEND_MODE', 'REFRESH', 'REFRESH_TIME', 'LOCKSIZE', 'VOLATILE', 'ROW_FORMAT', 'PROPERTY', 'STATISTICS_PROFILE', 'COMPRESSION', 'ROWCOMPMODE', 'ACCESS_MODE', 'CLUSTERED', 'ACTIVE_BLOCKS', 'DROPRULE', 'MAXFREESPACESEARCH', 'AVGCOMPRESSEDROWSIZE', 'AVGROWCOMPRESSIONRATIO', 'AVGROWSIZE', 'PCTROWSCOMPRESSED', 'LOGINDEXBUI